In [1]:
%load_ext autoreload
%autoreload 2

import cv2 as cv
import re
import matplotlib as mpl
import matplotlib.pyplot as plt

from qdanalysis.preprocessing.strokedecomposition import simple_stroke_segment
import pathlib

In [2]:
#to make generated figures easier to view
mpl.rcParams['figure.dpi']=300

In [3]:
#path_name = '../datasets/CERUG'
path_name = './CERUG'
path = pathlib.Path(path_name)
files = [fp for fp in path.iterdir()]
files

[PosixPath('CERUG/Writer0909_03-01.ppm'),
 PosixPath('CERUG/Writer0202_03-02.ppm'),
 PosixPath('CERUG/Writer0202_01.ppm')]

In [4]:
#associative arrays, temp
writers = []
pages = []
page_part = []
strokes = []

for file in path.iterdir():
    filename = file.stem
    print(filename)

    writer_no, page_no, page_part_no = re.search(r'(\d*)_(\d*)-?(\d*)?', filename).groups()
    writers.append(writer_no)
    pages.append(page_no)
    page_part.append(page_part_no)

    #open image and process strokes
    image = cv.imread(str(file), cv.IMREAD_GRAYSCALE)

    strokes.append(simple_stroke_segment(image))

Writer0909_03-01
Writer0202_03-02
Writer0202_01


In [6]:
writers

['0909', '0202', '0202']

In [7]:
pages

['03', '03', '01']

In [8]:
page_part

['01', '02', '']

In [34]:
import pandas as pd

In [45]:
stroke_sizes = []
for stroke_list in strokes:
    stroke_shapes = map(lambda x: x.shape, stroke_list)
    stroke_sizes.extend(list(x for x in stroke_shapes))

In [48]:
len(stroke_sizes)

5224

In [50]:
size_df = pd.DataFrame(stroke_sizes, columns=['height', 'width'])

In [52]:
size_df['height'].describe()

count    5224.000000
mean       16.997511
std        48.694979
min         1.000000
25%         7.000000
50%        12.000000
75%        21.000000
max      2693.000000
Name: height, dtype: float64

In [53]:
size_df['width'].describe()

count    5224.000000
mean       12.869640
std        12.787264
min         1.000000
25%         7.000000
50%        10.000000
75%        16.000000
max       481.000000
Name: width, dtype: float64

In [63]:
#temp solution to stroke decomp bug
from scipy.stats import zscore
import numpy as np

height_scores = zscore(size_df['height'])
width_scores = zscore(size_df['width'])
size_df_filtered = size_df[np.logical_and(np.abs(height_scores) < 3, np.abs(width_scores) < 3)]

5200

In [64]:
size_df_filtered['height'].describe()

count    5200.000000
mean       15.532885
std        11.356626
min         1.000000
25%         7.000000
50%        11.500000
75%        21.000000
max       142.000000
Name: height, dtype: float64

In [66]:
size_df_filtered['width'].describe()

count    5200.000000
mean       12.364423
std         7.066230
min         1.000000
25%         7.000000
50%        10.000000
75%        16.000000
max        51.000000
Name: width, dtype: float64